<a href="https://colab.research.google.com/github/zimzion/EYTIP/blob/master/Functional_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np

In [0]:
def pfactorsl(x):
  if x % 2 == 0:
    yield 2
    if x // 2 > 1:
      yield from pfactorsl(x // 2)
    return

  to = int(math.sqrt(x) + .5) + 1
  for i in range(3, to, 2):
    if x % i == 0:
      yield i
      if x // i > 1:
        yield from pfactorsl(x // i)
      return
  yield x

In [0]:
n = 1023
factors = list(pfactorsl(n))
n2 = np.prod(factors)
mul_str = ' X '.join([str(e) for e in factors])
print(f'{mul_str} = {n2} = {n} ({n==n2})')

3 X 11 X 31 = 1023 = 1023 (True)


In [0]:
import collections
from collections import namedtuple

Rank_Data = namedtuple("Rank_Data", ("rank_seq", "raw"))

In [0]:
def yield_sequence(rank, same_rank_iter):
  head = next(same_rank_iter)
  yield rank, head
  yield from yield_sequence(rank, same_rank_iter)

def ranker(sorted_iter, base, same_rank_seq, key):
  try:
    value = next(sorted_iter)
  except StopIteration:
    dups = len(same_rank_seq)
    yield from yield_sequence((base+1+base+dups)/2, iter(same_rank_seq))
    return

  if key(value.raw) == key(same_rank_seq[0].raw):
    yield from ranker(sorted_iter, base, same_rank_seq + [value], key)
  else:
    dups = len(same_rank_seq)
    yield from yield_sequence((base+1+base+dups)/2, iter(same_rank_seq))
    yield from ranker(sorted_iter, base+dups, [value], key)

def rerank(rank_data_collections, key):
  sorted_iter = iter(sorted(rank_data_collections, key=lambda obj : key(obj.raw)))
  head = next(sorted_iter)
  yield from ranker(sorted_iter, 0, [head], key)


In [0]:
def rank_data(data, key = lambda obj : obj):
  if isinstance(data, collections.abc.Iterator):
    yield from rank_data(tuple(data), key)
    return

  if not isinstance(data[0], Rank_Data):
    yield from rank_data((Rank_Data((), d) for d in data), key)
    return

  yield from (Rank_Data(rd.rank_seq + (r,), rd.raw) for r, rd in rerank(data, key))

In [39]:
scalars = [0.8, 1.2, 1.2, 2.3, 18]
list(rank_data(scalars))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: generator 'yield_sequence' raised StopIteration
  after removing the cwd from sys.path.


[Rank_Data(rank_seq=(1.0,), raw=0.8),
 Rank_Data(rank_seq=(2.5,), raw=1.2),
 Rank_Data(rank_seq=(2.5,), raw=1.2),
 Rank_Data(rank_seq=(4.0,), raw=2.3),
 Rank_Data(rank_seq=(5.0,), raw=18)]

In [40]:
pairs = ((2, 0.8), (3, 1.2), (5, 1.2), (7, 2.3), (11, 18))
rank_x = tuple(rank_data(pairs, key=lambda x : x[0]))
rank_x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: generator 'yield_sequence' raised StopIteration
  after removing the cwd from sys.path.


(Rank_Data(rank_seq=(1.0,), raw=(2, 0.8)),
 Rank_Data(rank_seq=(2.0,), raw=(3, 1.2)),
 Rank_Data(rank_seq=(3.0,), raw=(5, 1.2)),
 Rank_Data(rank_seq=(4.0,), raw=(7, 2.3)),
 Rank_Data(rank_seq=(5.0,), raw=(11, 18)))

In [41]:
rank_xy = tuple(rank_data(rank_x, key=lambda x : x[1]))
rank_xy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: generator 'yield_sequence' raised StopIteration
  after removing the cwd from sys.path.


(Rank_Data(rank_seq=(1.0, 1.0), raw=(2, 0.8)),
 Rank_Data(rank_seq=(2.0, 2.5), raw=(3, 1.2)),
 Rank_Data(rank_seq=(3.0, 2.5), raw=(5, 1.2)),
 Rank_Data(rank_seq=(4.0, 4.0), raw=(7, 2.3)),
 Rank_Data(rank_seq=(5.0, 5.0), raw=(11, 18)))